# Importações iniciais

In [51]:
import pandas as pd

In [52]:
ST = pd.read_csv("STSC202202.csv",low_memory=False)

## Tipos de leito

In [53]:
ST.rename(columns={"QTLEITP1": "leito_cirurgico", "QTLEITP2": "leito_clinico", "QTLEITP3": "leito_complementar"},inplace=True)

## Agregando dados

## Leitos gerais

In [54]:
leitos_columns = [x for x in ST.columns if x.startswith("QTLEIT")]
leitos = ST[leitos_columns].apply(sum, axis=1)
ST["leitos"] = leitos
ST.drop(labels=leitos_columns,axis="columns",inplace=True)

## Consultórios

In [55]:
consultorios_columns = [x for x in ST.columns if x.startswith("QTINST")]
consultorios = ST[consultorios_columns].apply(sum, axis=1)
ST["consultorios"] = consultorios
ST.drop(labels=consultorios_columns,axis="columns",inplace=True)

## Metadados

In [6]:
atendimento_hospitalar = {
    "URGEMERG": "URGÊNCIA/EMERGÊNCIA",
    "ATENDAMB": "ATENDIMENTO AMBULATORIAL",
    "CENTRCIR": "ATENDIMENTO HOSPITALAR-CENTRO CIRÚRGICO",
    "CENTROBS": "ATENDIMENTO HOSPITALAR-CENTRO OBSTÉTRICO",
    "CENTRNEO": "ATENDIMENTO HOSPITALAR-UNIDADE NEONATAL",
    "ATENDHOS": "ATENDIMENTO HOSPITALAR"
}

tipos_atendimento = {
    "AP01": "INTERNACAO",
    "AP02": "AMBULATORIAL",
    "AP03": "SADT",
    "AP04": "URGENCIA",
    "AP05": "OUTROS",
    "AP06": "VIGILANCIA",
    "AP07": "REGULACAO"
}

tipos_convênio = {
    "CV01": "SUS",
    "CV02": "PARTICULAR",
    "CV03": "PLANO-SEGURO PROPRIO",
    "CV04": "PLANO-SEGURO TERCEIRO",
    "CV05": "PLANO de SAUDE PUBLICO",
    "CV06": "SAUDE PRIVADO"
}



In [7]:
ATENDIMENTOS_CONVENIOS = {}
for key1, val1 in tipos_atendimento.items():
    for key2, val2 in tipos_convênio.items():
        ATENDIMENTOS_CONVENIOS[key1+key2] = f"{val1} ({val2})"

In [8]:
ATENDIMENTOS_CONVENIOS

{'AP01CV01': 'INTERNACAO (SUS)',
 'AP01CV02': 'INTERNACAO (PARTICULAR)',
 'AP01CV03': 'INTERNACAO (PLANO-SEGURO PROPRIO)',
 'AP01CV04': 'INTERNACAO (PLANO-SEGURO TERCEIRO)',
 'AP01CV05': 'INTERNACAO (PLANO de SAUDE PUBLICO)',
 'AP01CV06': 'INTERNACAO (SAUDE PRIVADO)',
 'AP02CV01': 'AMBULATORIAL (SUS)',
 'AP02CV02': 'AMBULATORIAL (PARTICULAR)',
 'AP02CV03': 'AMBULATORIAL (PLANO-SEGURO PROPRIO)',
 'AP02CV04': 'AMBULATORIAL (PLANO-SEGURO TERCEIRO)',
 'AP02CV05': 'AMBULATORIAL (PLANO de SAUDE PUBLICO)',
 'AP02CV06': 'AMBULATORIAL (SAUDE PRIVADO)',
 'AP03CV01': 'SADT (SUS)',
 'AP03CV02': 'SADT (PARTICULAR)',
 'AP03CV03': 'SADT (PLANO-SEGURO PROPRIO)',
 'AP03CV04': 'SADT (PLANO-SEGURO TERCEIRO)',
 'AP03CV05': 'SADT (PLANO de SAUDE PUBLICO)',
 'AP03CV06': 'SADT (SAUDE PRIVADO)',
 'AP04CV01': 'URGENCIA (SUS)',
 'AP04CV02': 'URGENCIA (PARTICULAR)',
 'AP04CV03': 'URGENCIA (PLANO-SEGURO PROPRIO)',
 'AP04CV04': 'URGENCIA (PLANO-SEGURO TERCEIRO)',
 'AP04CV05': 'URGENCIA (PLANO de SAUDE PUBLICO)',
 

In [9]:
for key, val in tipos_atendimento.items():
    colunas = [x for x in ST.columns if x.startswith(key)]
    ST["servico_" + val.lower()] = ST[colunas].apply(sum,axis=1)

In [10]:
programa = {
    "G1": "Atenção Básica Ambulatorial",
    "G2": "Média Complexidade Ambulatorial",
    "G3": "Cod 3",
    "G4": "Alta Complexidade Ambulatorial",
    "G5": "Média Complexidade Hospitalar",
    "G6": "Alta Complexidade Hospitalar"
}

PROGRAMAS = {}
for x, label in [("E", " (Estadual)"), ("M", " (Municipal)")]:
    for key, val in programa.items():
        PROGRAMAS["GESPR"+key+x] = val + label

In [11]:
PROGRAMAS

{'GESPRG1E': 'Atenção Básica Ambulatorial (Estadual)',
 'GESPRG2E': 'Média Complexidade Ambulatorial (Estadual)',
 'GESPRG3E': 'Cod 3 (Estadual)',
 'GESPRG4E': 'Alta Complexidade Ambulatorial (Estadual)',
 'GESPRG5E': 'Média Complexidade Hospitalar (Estadual)',
 'GESPRG6E': 'Alta Complexidade Hospitalar (Estadual)',
 'GESPRG1M': 'Atenção Básica Ambulatorial (Municipal)',
 'GESPRG2M': 'Média Complexidade Ambulatorial (Municipal)',
 'GESPRG3M': 'Cod 3 (Municipal)',
 'GESPRG4M': 'Alta Complexidade Ambulatorial (Municipal)',
 'GESPRG5M': 'Média Complexidade Hospitalar (Municipal)',
 'GESPRG6M': 'Alta Complexidade Hospitalar (Municipal)'}

In [12]:
from collections import Counter

In [13]:
colunas = [x for x in ST.columns if x.startswith("servico_")]
SERVICOS = {}
for x in colunas:
    SERVICOS[x] = Counter(ST[x].values)

In [14]:
SERVICOS

{'servico_internacao': Counter({0: 21174, 2: 97, 3: 71, 1: 75, 4: 33}),
 'servico_ambulatorial': Counter({2: 7181,
          1: 9983,
          0: 2603,
          3: 1438,
          4: 243,
          5: 2}),
 'servico_sadt': Counter({0: 18825, 2: 959, 1: 1182, 4: 106, 3: 377, 5: 1}),
 'servico_urgencia': Counter({0: 20797, 2: 137, 1: 421, 3: 66, 4: 29}),
 'servico_outros': Counter({0: 20407, 1: 957, 2: 81, 3: 5}),
 'servico_vigilancia': Counter({1: 684, 0: 20764, 2: 2}),
 'servico_regulacao': Counter({0: 20919, 1: 526, 2: 2, 4: 3})}

# Iterando sobre o dataset

In [33]:
metadados = pd.DataFrame(columns=["CNES", "key", "value"], dtype="category")

In [30]:
def append_metadados(CNES, key, val):
    global metadados
    new_val = pd.Series([CNES, key, val], index=["CNES", "key", "value"])
    metadados = pd.concat([metadados, new_val.to_frame().T], ignore_index=True)

In [34]:
metadados

,CNES,key,value


In [35]:
for i, reg in ST.iterrows(): 
    for key, val in ATENDIMENTOS_CONVENIOS.items():
        if reg[key] == 1:
            append_metadados(reg.CNES, "atendimentos", val)
    for key, val in PROGRAMAS.items():
        if reg[key] == 1:
            append_metadados(reg.CNES, "programa", val)
    for key, val in atendimento_hospitalar.items():
        if reg[key] == 1:
            append_metadados(reg.CNES, "hospital", val)

In [37]:
metadados.to_csv("metadados.csv", index=False)

In [66]:
metadados.to_pickle("metadados.pd.pkl")

In [56]:
interesse = ["CNES", "CODUFMUN", "COD_CEP",  "leito_cirurgico", "leito_clinico", "leito_complementar", "leitos", "consultorios" ]

In [65]:
ST[interesse].to_pickle("dados.pd.pkl")